In [1]:
%run ./../../utils/commonImports.py
%matplotlib inline

# Load data

In [2]:
db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data = load_trading_data(db, 'chart_data', from_date='2014-01-01', period='5min')
data.shape

(381242, 10)

# Savitzky - Golay filter - test

In [65]:
from scipy.signal import savgol_filter
x = [1, 1.5, 2, 3.0, 4, 5.5, 6, 7.6, 8, 9]
w = 5
w_half = int(w/2)
maxw = len(x)-w_half

In [66]:
res = [np.nan] * (w-1)
sg = savgol_filter(x, w, 1)
res += sg.tolist()[w_half:maxw]
np.round(res, 2)

array([ nan,  nan,  nan,  nan, 2.3 , 3.2 , 4.1 , 5.22, 6.22, 7.22])

In [67]:
np.round(sg_filter_polyfit(x, w, 1), 2)

array([ nan,  nan,  nan,  nan, 3.8 , 5.2 , 6.2 , 7.46, 8.24, 9.02])

In [68]:
sg

array([0.8 , 1.55, 2.3 , 3.2 , 4.1 , 5.22, 6.22, 7.22, 8.12, 9.02])

In [17]:
import time
def sg_filter_polyfit(x, window_size, deg):
    res = [np.nan] * (window_size-1)
    start = time.time()
    for i in range(window_size-1, len(x)):
        if i % 10**4 == 10**4 - 1:
            print(i, "%.2fs" % (time.time() - start))
            start = time.time()
        lasti = max(0, i-window_size+1)
        p = np.polyfit(range(window_size), x[lasti:i+1], deg=deg)
        y = np.polyval(p, window_size-1)
        res.append(y)
    return res

# SG filter implementation which don't use future values

In [ ]:
from IPython.display import clear_output
filtered = data.copy()
last_time = 0
output = "D:\\Dropbox\\My work\\krypl-project\\analysis\\sg-filter.tsv"

for w in [3, 5, 10]:
    for deg in [1, 2, 3, 4]:
        for col in ['close', 'open', 'high', 'low']:
            start = time.time()
            f_name = f'{col}_w{w}_deg{deg}'
            print(f_name, "last %.2fs" % last_time)
            filtered[f_name] = sg_filter_polyfit(data[col], w, deg)
            last_time = time.time() - start
            write_tsv(output)
            clear_output()